In [1]:
def naive_search(pattern, text):  # O(NM)
    m = len(pattern)
    n = len(text)

    # this operation takes O(n)
    for i in range(n-m+1):
        # track the letters in the pattern (starting 0) from left to right
        j = 0

        # all the letters of the pattern O(m) - in worst-case this approach takes O(n*m)
        while j < m:
            # we have to compare the characters
            if text[i+j] != pattern[j]:
                break
            # consider the next characters
            j = j + 1
        if j == m:
            print('Found a match at index %s' % i)

In [2]:
naive_search('abcd', 'abcdeefabcabcd')

Found a match at index 0
Found a match at index 10


## Rabin-Karp Substring Search

- It has O(M+N), but its worst case is still O(MN)
- The aim is to compare the pattern and the region of the text with a single test - in O(1) constant running time
- To reduce spurious, make a good hash function is critical to get the good performance
    - Rabin fingerprint funtion uses a polynomial
    - Rolling Hashing function calculate the hash value based on the previous value -> O(1)
        - h('bbb') = h('abb') - h('a') + h('b')
        - ((1x26^2 + 2x26^1 + 2x26^0) - 1x26^2) x 26 + 2x26^0

In [1]:

class RabinKarp:

    def __init__(self, pattern, text):
        self.pattern = pattern
        self.text = text        
        self.d = 26  # the size of the alphabet (26)        
        self.q = 31  # prime number for the (%) modulo operator

    def search(self):

        m = len(self.pattern)
        n = len(self.text)

        # hashes for the region of text and the pattern
        hash_text = 0
        hash_pattern = 0

        # the largest polynomial term in the fingerprint function
        h = 1
        for _ in range(m-1):
            h = (h*self.d) % self.q

        # pre-compute the hash of the pattern O(M)
        for i in range(m):
            hash_pattern = (self.d*hash_pattern + ord(self.pattern[i])) % self.q
            hash_text = (self.d * hash_text + ord(self.text[i])) % self.q

        # slide the pattern over text one by one
        for i in range(n-m+1):

            # check the hash values of current window of text
            # and pattern. If the hash values match then only
            # check for characters on by one
            if hash_text == hash_pattern:

                # naive approach to check the characters
                j = 0

                while j < m:
                    if self.text[i+j] != self.pattern[j]:
                        break

                    j = j + 1

                if j == m:
                    print("Found match at index %s" % i)

            # update the hash for the actual substring of the text
            # apply the rolling hash approach
            if i < n-m:
                hash_text = ((hash_text - ord(self.text[i]) * h) * self.d + ord(self.text[i + m])) % self.q

                # we might get negative value so we have to make sure the hash is positive
                if hash_text < 0:
                    hash_text = hash_text + self.q


In [2]:
algorithm = RabinKarp('abcd', 'abcdabcbcabcd')
algorithm.search()

Found match at index 0
Found match at index 9


## Knuth-Morris-Pratt (KMP) Algorithm
- KMP is O(N+M) linear running time even in the worst-case scenario.
- The algorithm must proprocess the pattern with O(M) running time complexity and with O(M) additional memory complexity
    - Construct "partial match table" (or failture function)
- Analyze the prefix and the suffix of the pattern
- KMP checks whether some prefixes are matching any suffixes in the pattern.
    - It looks for the longest prefix which is the same as some suffixes. 
- The $\pi$(p) encapsulates the knowlege about how the pattern matches against the shifts of itself. This information can be used to avoid a useless shift of the P pattern

In [3]:

def construct_pi(pattern):

    # the table has as many values as the length of the pattern (first item is always a 0)
    pi_table = [0]*len(pattern)

    prefix_counter = 0
    i = 1

    # O(M) linear running time
    while i < len(pattern):
        if pattern[i] == pattern[prefix_counter]:
            prefix_counter = prefix_counter + 1
            pi_table[i] = prefix_counter
            i = i + 1
        else:
            if prefix_counter != 0:
                prefix_counter = pi_table[prefix_counter-1]
            else:
                pi_table[i] = 0
                i = i + 1

    return pi_table


def search(text, pattern):

    pi_table = construct_pi(pattern)
    # index i tracks the text - index j tracks the pattern
    i = 0
    j = 0

    # we have to iterate until the i index is less than the N length of the text
    # and we have to make sure j is smaller than the M length of the pattern
    while i < len(text) and j < len(pattern):
        # if the letters are matching we increment both indexes
        if text[i] == pattern[j]:
            i = i + 1
            j = j + 1
        # we found the pattern in the text (+ reinitialize the j index to be able find more patterns)
        if j == len(pattern):
            print('Pattern found at index %s' % (i-j))
            j = pi_table[j - 1]
        # if there is a mismatch
        elif i < len(text) and text[i] != pattern[j]:
            # if we can decrement j then we decrement it based on the pi table
            if j != 0:
                j = pi_table[j-1]
            # if we are not able to decrement the j (because it has value 0) we increment i
            else:
                i = i + 1

In [4]:
search('aacaabaab', 'aab')

Pattern found at index 3
Pattern found at index 6


## Z Substring Search
- State of Art
- Z algorithm has O(N+M) linear running time compleixty and it needs additional O(N+M) memory.
- So, it is not an in place algorithm
- Z<sub>i</sub> value is the length of the longest substring in S starting at position i that matches a prefix of S
    - If we have the Z<sub>i</sub> values then we can find all the P pattern matches in the original S text. 
    - We just have to find the values in the Z table with >= |P| so the lengt of the P pattern
- Z box: the Z<sub>i</sub> values are greater than 0. 


In [5]:
class ZAlgorithm:

    def __init__(self, pattern, text):
        self.pattern = pattern
        self.text = text
        # we have to concatenate the pattern and the text
        self.S = pattern + text
        # int table for the Z values
        self.Z = [0 for _ in range(len(self.S))]

    def construct_z_table(self):

        # the first item (index 0) is the length of the S
        self.Z[0] = len(self.S)

        # the first and last items in the Z box
        left = 0
        right = 0

        # consider all the letters of the S string (starting with index 1)
        for k in range(1, len(self.S)):
            # we are not within a Z box (naive approach) CASE I
            if k > right:

                n = 0

                while n+k < len(self.S) and self.S[n] == self.S[n+k]:
                    n = n + 1

                self.Z[k] = n

                if n > 0:
                    left = k
                    right = k + n - 1
            else:
                # we are inside a Z box so maybe we can copy the values
                p = k - left
                # case II when we can copy the values of Z
                if self.Z[p] < right - k + 1:
                    self.Z[k] = self.Z[p]
                else:
                    # we can not copy the values (case III)
                    i = right + 1

                    while i < len(self.S) and self.S[i] == self.S[i - k]:
                        i = i + 1

                    self.Z[k] = i - k
                    left = k
                    right = i - 1

    def search(self):

        self.construct_z_table()

        # we just have to consider the values in the Z table in O(N+M)
        for i in range(1, len(self.Z)):
            if self.Z[i] >= len(self.pattern):
                print("Match found at index %s" % (i - len(self.pattern)))

In [6]:
algorithm = ZAlgorithm('aabza', 'abzcaabzaabza')
algorithm.search()

Match found at index 4
Match found at index 8


## Substring Search Comparison

- 